# Pandas Dataframe Agent

This notebook shows how to use agents to interact with a pandas dataframe. It is mostly optimized for question answering.

**NOTE: this agent calls the Python agent under the hood, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.**

In [1]:
import json
filepath = 'credentials.json'
file = open(filepath, 'r')

# Open the credentials file with json.load
credentials = json.load(file)
file.close()

In [2]:
!pip install langchain
!pip install pandas
!pip install openpyxl
!pip install openai
!pip install tabulate

In [56]:
import asyncio
from pyppeteer import launch
from typing import List, Dict
from bs4 import BeautifulSoup

# Helper function to extract content based on type
async def get_content_for_type(element, page, selector: str, type_: str) -> str:
    val = ""
    text_ob = element
    try:
        text_ob = await element.querySelector(selector)
    except Exception as e:
        print("INFO: selector not valid - probably the parent object is the object")

    try:
        if type_ == 'TEXT':
            val = (await page.evaluate('(el) => el.textContent', text_ob)).strip()
        elif type_ == 'IMAGE':
            # Probably not complete - could also be in srcset or so....
            val = await page.evaluate('(el) => el.src', text_ob)
        elif type_ == 'LINK':
            # Probably not complete
            val = await page.evaluate('(el) => el.href', text_ob)

        return val
    except Exception as e:
        print("INFO: object not found", e)


# Function to generate a common selector
def generate_common_selector(selectors):
    arr = [s.replace(' > ', '> ').split(' ') for s in selectors]
    arr.sort()
    a1 = arr[0]
    a2 = arr[len(arr) - 1]
    L = len(a1)
    i = 0
    while i < L and a1[i] == a2[i]:
        i += 1
    return ' '.join([s.replace('>', ' >') for s in a1[:i]])


# Function to scrape data from the website
async def scrape_data(page, selectors: List[Dict]) -> List[Dict]:
    common_sub_path = generate_common_selector([s["selector"] for s in selectors])
    sub_selectors = [
        {**s, "selector": s["selector"].replace(common_sub_path, "").strip()} for s in selectors
    ]
    common_sub_path = common_sub_path[:-2] if common_sub_path.endswith('>') else common_sub_path

    elements = await page.querySelectorAll(common_sub_path) if common_sub_path else [page]

    scraped_data = []
    for element in elements:
        data = {}
        for selector in sub_selectors:
            data_point = await get_content_for_type(element, page, selector["selector"], selector["type"])
            if data_point:
                data[selector["name"]] = data_point
        scraped_data.append(data)

    return scraped_data


async def scrape_general_data(page):
    """Load the specified URLs using Playwright and parse using BeautifulSoup."""

    results = ""
    page_source = await page.content()
    soup = BeautifulSoup(page_source, "html.parser")

    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    results = "\n".join(chunk for chunk in chunks if chunk)
    return results


SELECTORS = [{"name":"drugName","description":"drugName","selector":".headline a","type":"TEXT"},{"name":"link","description":"link","selector":".headline a","type":"LINK"}];

LINK = "https://www.centerwatch.com/directories/1067-fda-approved-drugs"


async def main(link):
    browser = await launch(
        headless=True,
        timeout=100000,
        ignoreDefaultArgs=["--enable-automation"],
        args=[],
        defaultViewport=None
    )

    page = await browser.newPage()

    await page.goto(link, waitUntil=["networkidle2"], timeout=15000)

    scraped_data = await scrape_data(page, SELECTORS)

    print(scraped_data)
    await browser.close()
    return scraped_data

async def scrape_main(link):
    browser = await launch(
        headless=True,
        timeout=100000,
        ignoreDefaultArgs=["--enable-automation"],
        args=[],
        defaultViewport=None
    )

    page = await browser.newPage()

    await page.goto(link, waitUntil=["networkidle2"], timeout=15000)

    scraped_data = await scrape_general_data(page)

    print(scraped_data)
    await browser.close()
    return scraped_data


# scraped_data = asyncio.get_event_loop().run_until_complete(main(LINK))

INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent object is the object
INFO: selector not valid - probably the parent o

In [59]:
scraped_data = asyncio.get_event_loop().run_until_complete(scrape_main(LINK))


FDA Approved Drugs | CenterWatch
Sign In
Create Account
Sign Out
My Account
0 itemsSKIP TO CONTENTSKIP NAVIGATIONPatient ResourcesProfessional ResourcesWhite PapersClinical Trial ListingsAdvertiseCOVID-19Patient ResourcesCOVID-19 Patient Resource CenterClinical Trial ListingsWhat is Clinical Research?Volunteering for a Clinical TrialUnderstanding Informed ConsentUseful ResourcesFDA Approved DrugsProfessional ResourcesResearch Center ProfilesMarket ResearchFDA Approved DrugsTraining GuidesBookseLearningEventsNewslettersWhite PapersSOPseCFR and Guidances
LabelSearchSKIP TO CONTENTSKIP NAVIGATIONPatient ResourcesCOVID-19 Patient Resource CenterClinical Trial ListingsWhat is Clinical Research?Volunteering for a Clinical TrialUnderstanding Informed ConsentUseful ResourcesFDA Approved DrugsProfessional ResourcesResearch Center ProfilesMarket ResearchFDA Approved DrugsTraining GuidesBookseLearningEventsNewslettersWhite PapersSOPseCFR and GuidancesWhite PapersClinical Trial ListingsAdvertiseCO

'VID-19\nSign In\nCreate Account\nSign Out\nMy Account\n0 items\nHome » Directories » FDA Approved Drugs\nFDA Approved Drugs\nThe following database contains a listing of drugs approved by the Food and Drug Administration (FDA) for sale in the United States. Drug information typically includes the drug name, approval status, indication of use, and clinical trial results.\nAND\nSearch\nCondition\nTherapeutic Area\nCompany\nApproval Year\nFilter by drug name:\nA\nB\nC\nD\nE\nF\nG\nH\nI\nJ\nK\nL\nM\nN\nO\nP\nQ\nR\nS\nT\nU\nV\nW\nX\nY\nZ\nAbcema (idecabtagene vicleucel)\nAbilify (aripiprazole)\nAbraxane (paclitaxel protein-bound particles for injectable suspension)\nAbstral (fentanyl sublingual tablets)\nAbthrax (raxibacumab)\nAccolate (zafirlukast)\nAciphex (rabeprazole sodium)\nActemra (tocilizumab)\nActivella (Estradiol/Norethindrone Acetate) Tablets\nActonel\nActos (pioglitazone); ACTOplus met (pioglitazone and metformin HCl)\nAcular (ketorolac tromethamine) ophthalmic solution\nAcuvai

In [62]:
(scraped_data[4000:])

'king some types of cookies may impact your experience of the site and the services we are able to offer.\nPrivacy PolicyAllow All Manage Consent PreferencesTargeting Cookies\nTargeting Cookies These cookies may be set through our site by our advertising partners. They may be used by those companies to build a profile of your interests and show you relevant adverts on other sites. They do not store directly personal information, but are based on uniquely identifying your browser and internet device. If you do not allow these cookies, you will experience less targeted advertising.Cookies Details\u200eStrictly Necessary CookiesAlways ActiveThese cookies are necessary for the website to function and cannot be switched off in our systems. They are usually only set in response to actions made by you which amount to a request for services, such as setting your privacy preferences, logging in or filling in forms. You can set your browser to block or alert you about these cookies, but some par

In [81]:
data = await main()

In [82]:
data

[{'drugName': 'Abcema (idecabtagene vicleucel)',
  'link': 'https://www.centerwatch.com/directories/1067-fda-approved-drugs/listing/4697-abcema-idecabtagene-vicleucel'},
 {'drugName': 'Abilify (aripiprazole)',
  'link': 'https://www.centerwatch.com/directories/1067-fda-approved-drugs/listing/3060-abilify-aripiprazole'},
 {'drugName': 'Abraxane (paclitaxel protein-bound particles for injectable suspension)',
  'link': 'https://www.centerwatch.com/directories/1067-fda-approved-drugs/listing/3061-abraxane-paclitaxel-protein-bound-particles-for-injectable-suspension'},
 {'drugName': 'Abstral (fentanyl sublingual tablets)',
  'link': 'https://www.centerwatch.com/directories/1067-fda-approved-drugs/listing/3063-abstral-fentanyl-sublingual-tablets'},
 {'drugName': 'Abthrax (raxibacumab)',
  'link': 'https://www.centerwatch.com/directories/1067-fda-approved-drugs/listing/3064-abthrax-raxibacumab'},
 {'drugName': 'Accolate (zafirlukast)',
  'link': 'https://www.centerwatch.com/directories/1067-

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool


prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

In [84]:
import os
import yaml

from langchain.agents import (
    create_json_agent,
    AgentExecutor
)
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

In [85]:
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=llm,
    toolkit=json_toolkit,
    verbose=True
)

In [97]:
json_spec = JsonSpec(dict_=str(data), max_value_length=4000)
json_spec

ValidationError: 1 validation error for JsonSpec
dict_
  value is not a valid dict (type=type_error.dict)

In [98]:
dict(str(data[0:2]))

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [89]:
json_agent_executor.run("How many drugName start with A?")

'Agent stopped due to iteration limit or time limit.'

In [ ]:
json_agent_executor.run("When you search for the link for Adderall XR, what does it desscribe about the drug?")

In [34]:
import os
import pandas as pd
from langchain.experimental.autonomous_agents.autogpt.agent import AutoGPT
from langchain.chat_models import ChatOpenAI

from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain.docstore.document import Document
import asyncio
import nest_asyncio
# Needed synce jupyter runs an async eventloop
nest_asyncio.apply()

In [7]:
llm = ChatOpenAI(temperature=1.0, openai_api_key=credentials["OPENAI_API_KEY"])

In [18]:
import os
from contextlib import contextmanager
from typing import Optional
from langchain.agents import tool
from langchain.tools.file_management.read import ReadFileTool
from langchain.tools.file_management.write import WriteFileTool

ROOT_DIR = "./data/"
os.makedirs(ROOT_DIR)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './data/'

In [35]:
@contextmanager
def pushd(new_dir):
    """Context manager for changing the current working directory."""
    prev_dir = os.getcwd()
    os.chdir(new_dir)
    try:
        yield
    finally:
        os.chdir(prev_dir)

@tool
def process_csv(
    csv_file_path: str, instructions: str, output_path: Optional[str] = None
) -> str:
    """Process a CSV by with pandas in a limited REPL.\
 Only use this after writing data to disk as a csv file.\
 Any figures must be saved to disk to be viewed by the human.\
 Instructions should be written in natural language, not code. Assume the dataframe is already loaded."""
    with pushd(ROOT_DIR):
        try:
            df = pd.read_csv(csv_file_path)
        except Exception as e:
            return f"Error: {e}"
        agent = create_pandas_dataframe_agent(llm, df, max_iterations=30, verbose=True)
        if output_path is not None:
            instructions += f" Save output to disk at {output_path}"
        try:
            result = agent.run(instructions)
            return result
        except Exception as e:
            return f"Error: {e}"

In [38]:
!playwright install

In [45]:
async def async_load_playwright(url: str) -> str:
    """Load the specified URLs using Playwright and parse using BeautifulSoup."""
    from bs4 import BeautifulSoup
    from playwright.async_api import async_playwright

    results = ""
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        try:
            page = await browser.new_page()
            await page.goto(url)

            page_source = await page.content()
            soup = BeautifulSoup(page_source, "html.parser")

            for script in soup(["script", "style"]):
                script.extract()

            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            results = "\n".join(chunk for chunk in chunks if chunk)
        except Exception as e:
            results = f"Error: {e}"
        await browser.close()
    return results

def run_async(coro):
    event_loop = asyncio.get_event_loop()
    return event_loop.run_until_complete(coro)

@tool
def browse_web_page(url: str) -> str:
    """Verbose way to scrape a whole webpage. Likely to cause issues parsing."""
    return async_load_playwright(url)

In [ ]:
from langchain.tools import BaseTool, DuckDuckGoSearchRun
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pydantic import Field
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain, BaseCombineDocumentsChain

In [50]:
res = asyncio.get_event_loop().run_until_complete(async_load_playwright("google.com"))

Task exception was never retrieved
future: <Task finished name='Task-255' coro=<Connection.run() done, defined at c:\Users\srava\anaconda3\envs\autogpt\Lib\site-packages\playwright\_impl\_connection.py:258> exception=NotImplementedError()>
Traceback (most recent call last):
  File "c:\Users\srava\anaconda3\envs\autogpt\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\srava\AppData\Local\Temp\ipykernel_4128\2225362524.py", line 1, in <module>
    res = asyncio.get_event_loop().run_until_complete(async_load_playwright("google.com"))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\srava\anaconda3\envs\autogpt\Lib\site-packages\nest_asyncio.py", line 90, in run_until_complete
    return f.result()
           ^^^^^^^^^^
  File "c:\Users\srava\anaconda3\envs\autogpt\Lib\asyncio\futures.py", line 203, in result
    raise self._exception.

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\srava\anaconda3\envs\autogpt\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\srava\AppData\Local\Temp\ipykernel_4128\2225362524.py", line 1, in <module>
    res = asyncio.get_event_loop().run_until_complete(async_load_playwright("google.com"))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\srava\anaconda3\envs\autogpt\Lib\site-packages\nest_asyncio.py", line 90, in run_until_complete
    return f.result()
           ^^^^^^^^^^
  File "c:\Users\srava\anaconda3\envs\autogpt\Lib\asyncio\futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\Users\srava\anaconda3\envs\autogpt\Lib\asyncio\tasks.py", line 267, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\Users\srava\AppData\Local\Temp\ipykernel_4128\3

In [49]:
res

<coroutine object async_load_playwright at 0x00000262841B1690>

In [21]:
from langchain.tools import BaseTool, DuckDuckGoSearchRun
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pydantic import Field
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain, BaseCombineDocumentsChain

In [22]:
def _get_text_splitter():
    return RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size = 500,
        chunk_overlap  = 20,
        length_function = len,
    )


class WebpageQATool(BaseTool):
    name = "query_webpage"
    description = "Browse a webpage and retrieve the information relevant to the question."
    text_splitter: RecursiveCharacterTextSplitter = Field(default_factory=_get_text_splitter)
    qa_chain: BaseCombineDocumentsChain
    
    def _run(self, url: str, question: str) -> str:
        """Useful for browsing websites and scraping the text information."""
        result = browse_web_page.run(url)
        docs = [Document(page_content=result, metadata={"source": url})]
        web_docs = self.text_splitter.split_documents(docs)
        results = []
        # TODO: Handle this with a MapReduceChain
        for i in range(0, len(web_docs), 4):
            input_docs = web_docs[i:i+4]
            window_result = self.qa_chain({"input_documents": input_docs, "question": question}, return_only_outputs=True)
            results.append(f"Response from window {i} - {window_result}")
        results_docs = [Document(page_content="\n".join(results), metadata={"source": url})]
        return self.qa_chain({"input_documents": results_docs, "question": question}, return_only_outputs=True)
    
    async def _arun(self, url: str, question: str) -> str:
        raise NotImplementedError

In [ ]:
from langchain.tools import BaseTool, DuckDuckGoSearchRun
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pydantic import Field
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain, BaseCombineDocumentsChain

In [23]:
query_website_tool = WebpageQATool(qa_chain=load_qa_with_sources_chain(llm))

In [117]:
!pip install faiss-cpu

In [28]:
# Memory
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools.human.tool import HumanInputRun

embeddings_model = OpenAIEmbeddings(openai_api_key=credentials["OPENAI_API_KEY"])
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [120]:
# !pip install duckduckgo_search
web_search = DuckDuckGoSearchRun()

In [24]:
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(root_dir="./data"),
    ReadFileTool(root_dir="./data"),
    process_csv,
    query_website_tool,
    # HumanInputRun(), # Activate if you want the permit asking for help from the human
]

In [160]:
!pip install google-search-results

In [9]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper(serpapi_api_key=credentials["SERPAPI_API_KEY"],)
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [25]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=llm,
    memory=vectorstore.as_retriever(search_kwargs={"k": 8}),
    # human_in_the_loop=True, # Set to True if you want to add feedback at each step.
)
agent.chain.set_verbose(True)

True

In [31]:
agent.run(["What is the latest SEC data about the top pharamaceutical companies with the most recent clinically approved drugs on the market. Write the collected data as well as clinical trial results to a local CSV file"])

My apologies. Here is the response:

```
{
    "thoughts": {
        "text": "",
        "reasoning": "Now that I have the new table with just the top five pharmaceutical companies that have the most clinically approved drugs, I will use the 'write_file' command to generate a text file summarizing the companies, their drug counts, and their SEC filing codes.",
        "plan": "- Use the write_file command to generate a text file summarizing the companies, their drug counts, and their SEC filing codes.",
        "criticism": "",
        "speak": "To generate a text file summarizing the top five pharmaceutical companies that have the most clinically approved drugs, their drug counts, and their SEC filing codes, I will use the write_file command."
    },
    "command": {
        "name": "write_file",
        "args": {
            "file_path": "/path/to/companies_summary.txt",
            "text": "Top five Pharmaceutical Companies with Most Clinically Approved Drugs:\nCompany Name, Approve

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jDLuuEJzRhFF9SVHdRIAMeN2 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jDLuuEJzRhFF9SVHdRIAMeN2 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

My apologies again for the error. Below is the response:

```
{
    "thoughts": {
        "text": "",
        "reasoning": "Now that I have the new table with just the top five pharmaceutical companies that have the most clinically approved drugs, I will use the 'write_file' command to generate a text file summarizing the companies, their drug counts, and their SEC filing codes.",
        "plan": "- Use the write_file command to generate a text file summarizing the companies, their drug counts, and their SEC filing codes.",
        "criticism": "",
        "speak": "To generate a text file summarizing the top five pharmaceutical companies that have the most clinically approved drugs, their drug counts, and their SEC filing codes, I will use the write_file command."
    },
    "command": {
        "name": "write_file",
        "args": {
            "file_path": "/path/to/companies_summary.txt",
            "text": "Top five Pharmaceutical Companies with Most Clinically Approved Drugs:\n

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jDLuuEJzRhFF9SVHdRIAMeN2 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jDLuuEJzRhFF9SVHdRIAMeN2 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

My apologies for the errors. Here is the correct response:

```
{
    "thoughts": {
        "text": "",
        "reasoning": "Now that I have the new table with just the top five pharmaceutical companies that have the most clinically approved drugs, I will use the 'write_file' command to generate a text file summarizing the companies, their drug counts, and their SEC filing codes.",
        "plan": "- Use the write_file command to generate a text file summarizing the companies, their drug counts, and their SEC filing codes.",
        "criticism": "",
        "speak": "To generate a text file summarizing the top five pharmaceutical companies that have the most clinically approved drugs, their drug counts, and their SEC filing codes, I will use the write_file command."
    },
    "command": {
        "name": "write_file",
        "args": {
            "file_path": "/path/to/companies_summary.txt",
            "text": "Top five Pharmaceutical Companies with Most Clinically Approved Drugs:

...

{
    "thoughts": {
        "text": "",
        "reasoning": "Since there was an error with the `query_webpage` command, I will need to use a different strategy to collect the required information. I will use the `search` command to find research papers or news articles about each of the top 5 pharmaceutical companies with clinically approved drugs, and scrape the necessary SEC data. I will then use the `write_file` command to append this data to the `top_pharmaceutical_companies.csv` file. Because the clinical trial data is often reported in multiple locations and databases, I will investigate the difference sources of clinical trial data using the `search` command and add the data manually by updating the `top_pharmaceutical_companies.csv` file. ",
        "plan": "- Use the `search` command to find research papers and news articles about the top 5 pharmaceutical companies with clinically approved drugs.\\n- Scrape the necessary SEC data.\\n- Append the SEC data to the `top_pharmaceu

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jDLuuEJzRhFF9SVHdRIAMeN2 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


{
    "thoughts": {
        "text": "",
        "reasoning": "The article does not contain the SEC data that is required. Thus, I will use the `search` command to look for the information from other sources. I will continue looking for research articles and SEC reports that contain the necessary information and store that information in a separate file.",
        "plan": "- Use the `search` command to find more research articles and SEC reports that contain the necessary SEC data of AstraZeneca, Pfizer, Roche, GlaxoSmithKline, and Johnson & Johnson.\\n- Store the collected information in a separate file.",
        "criticism": "",
        "speak": "The article found using the `search` command did not contain any relevant SEC data about the top 5 pharmaceutical companies. I will continue searching for SEC reports and research articles to find the necessary data and store it in a separate file."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input":

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jDLuuEJzRhFF9SVHdRIAMeN2 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jDLuuEJzRhFF9SVHdRIAMeN2 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

KeyboardInterrupt: 

In [ ]:
agent.run(["What were the winning boston marathon times for the past 5 years (ending in 2022)? Generate a table of the year, name, country of origin, and times."])